# Economic Evaluation and Reserves Estimation

This notebook demonstrates economic analysis of oil well production.

## What You'll Learn
- Calculate EUR (Estimated Ultimate Recovery)
- Compute NPV and cash flows
- Determine payback periods
- Run sensitivity analysis
- Create tornado plots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from decline_curve import dca
from decline_curve.models import ArpsParams

# Configure logging
import logging
from decline_curve.logging_config import configure_logging, get_logger

configure_logging(level=logging.INFO)
logger = get_logger(__name__)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline


## 1. Define Well Parameters

We'll use typical parameters for a Bakken shale oil well.

In [ ]:
# Well decline parameters
params = ArpsParams(
    qi=1200,   # Initial production rate (bbl/month)
    di=0.15,   # Nominal decline rate (1/month)
    b=0.4      # Hyperbolic exponent
)

logger.info("Well Parameters:")
logger.info(f"  Initial Rate (qi): {params.qi} bbl/month")
logger.info(f"  Decline Rate (di): {params.di:.3f} /month ({params.di*12:.2f} /year)")
logger.info(f"  b-factor: {params.b}")

# Economic parameters
oil_price = 70.0      # $/bbl
opex = 20.0           # Operating cost ($/bbl)
discount_rate = 0.10  # 10% annual discount rate
econ_limit = 10.0     # Economic limit (bbl/month)

logger.info("Economic Parameters:")
logger.info(f"  Oil Price: ${oil_price}/bbl")
logger.info(f"  Operating Cost: ${opex}/bbl")
logger.info(f"  Net Revenue: ${oil_price - opex}/bbl")
logger.info(f"  Discount Rate: {discount_rate*100}%")
logger.info(f"  Economic Limit: {econ_limit} bbl/month")

## 2. Calculate Reserves (EUR)

In [ ]:
# Generate forecast and calculate EUR
reserves = dca.reserves(params, t_max=240, econ_limit=econ_limit)

logger.info("Reserves Estimation:")
logger.info(f"  EUR (Estimated Ultimate Recovery): {reserves['eur']:,.0f} bbl")
logger.info(f"  Economic life: {len(reserves['t_valid'])} months")
logger.info(f"  Final production rate: {reserves['q_valid'][-1]:.1f} bbl/month")

# Visualize production forecast
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(reserves['t'], reserves['q'], 'b-', linewidth=2, label='Forecast Rate')
ax.plot(reserves['t_valid'], reserves['q_valid'], 'g-', linewidth=3, label='Economic Production')
ax.axhline(y=econ_limit, color='r', linestyle='--', linewidth=2, label=f'Economic Limit ({econ_limit} bbl/mo)')
ax.set_xlabel('Time (months)', fontsize=12)
ax.set_ylabel('Production Rate (bbl/month)', fontsize=12)
ax.set_title('Production Forecast and Economic Limit', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Economic Analysis

In [ ]:
# Calculate economic metrics
economics = dca.economics(
    production=pd.Series(reserves['q_valid']),
    price=oil_price,
    opex=opex,
    discount_rate=discount_rate
)

logger.info("Economic Metrics:")
logger.info(f"  NPV (Net Present Value): ${economics['npv']:,.0f}")
logger.info(f"  Undiscounted Cash Flow: ${economics['cash_flow'].sum():,.0f}")
logger.info(f"  Payback Period: {economics['payback_month']} months")

# Calculate additional metrics
total_revenue = reserves['eur'] * oil_price
total_opex = reserves['eur'] * opex
net_revenue = total_revenue - total_opex

logger.info(f"Revenue Breakdown:")
logger.info(f"  Gross Revenue: ${total_revenue:,.0f}")
logger.info(f"  Total OPEX: ${total_opex:,.0f}")
logger.info(f"  Net Revenue (undiscounted): ${net_revenue:,.0f}")

## 4. Cash Flow Analysis

In [ ]:
# Plot cash flow over time
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Monthly cash flow
months = np.arange(len(economics['cash_flow']))
ax1.bar(months, economics['cash_flow'], color='green', alpha=0.7)
ax1.set_xlabel('Month', fontsize=12)
ax1.set_ylabel('Monthly Cash Flow ($)', fontsize=12)
ax1.set_title('Monthly Cash Flow', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Cumulative cash flow
cumulative_cf = np.cumsum(economics['cash_flow'])
ax2.plot(months, cumulative_cf, 'b-', linewidth=2)
ax2.axhline(y=0, color='r', linestyle='--', linewidth=2)
if economics['payback_month'] is not None:
    ax2.axvline(x=economics['payback_month'], color='g', linestyle='--', 
                linewidth=2, label=f"Payback: Month {economics['payback_month']}")
    ax2.legend()
ax2.set_xlabel('Month', fontsize=12)
ax2.set_ylabel('Cumulative Cash Flow ($)', fontsize=12)
ax2.set_title('Cumulative Cash Flow', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Sensitivity Analysis

Analyze how NPV changes with different oil prices.

In [ ]:
# Test different price scenarios
prices = np.arange(40, 101, 10)
npv_results = []
eur_results = []

for price in prices:
    econ = dca.economics(
        production=pd.Series(reserves['q_valid']),
        price=price,
        opex=opex,
        discount_rate=discount_rate
    )
    npv_results.append(econ['npv'])
    eur_results.append(reserves['eur'])

# Create sensitivity plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(prices, npv_results, 'o-', linewidth=2, markersize=8, color='blue')
ax.axhline(y=0, color='r', linestyle='--', linewidth=2, alpha=0.5)
ax.axvline(x=oil_price, color='g', linestyle='--', linewidth=2, alpha=0.5, label=f'Base Case (${oil_price}/bbl)')
ax.set_xlabel('Oil Price ($/bbl)', fontsize=12)
ax.set_ylabel('NPV ($)', fontsize=12)
ax.set_title('NPV Sensitivity to Oil Price', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
plt.tight_layout()
plt.show()

# Find breakeven price
for i, (price, npv) in enumerate(zip(prices, npv_results)):
    if npv > 0:
        breakeven = price
        break
else:
    breakeven = None

if breakeven:
    logger.info(f"Breakeven oil price: ~${breakeven}/bbl")

## 6. Multi-Parameter Sensitivity

In [ ]:
# Define parameter grid for sensitivity analysis
param_grid = [
    (1000, 0.12, 0.3),  # Low case
    (1200, 0.15, 0.4),  # Base case
    (1400, 0.18, 0.5),  # High case
]

price_scenarios = [50, 60, 70, 80, 90]

# Run sensitivity analysis
sensitivity = dca.sensitivity_analysis(
    param_grid=param_grid,
    prices=price_scenarios,
    opex=opex,
    discount_rate=discount_rate,
    t_max=240,
    econ_limit=econ_limit
)

logger.info("Sensitivity Analysis Results:")
logger.info(sensitivity[['qi', 'di', 'b', 'price', 'EUR', 'NPV']].round(0))

In [ ]:
# Visualize sensitivity results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# EUR by initial rate and price
for price in price_scenarios:
    subset = sensitivity[sensitivity['price'] == price]
    ax1.plot(subset['qi'], subset['EUR'], 'o-', label=f'${price}/bbl', linewidth=2, markersize=6)

ax1.set_xlabel('Initial Rate (qi) bbl/month', fontsize=12)
ax1.set_ylabel('EUR (bbl)', fontsize=12)
ax1.set_title('EUR Sensitivity', fontsize=14, fontweight='bold')
ax1.legend(title='Oil Price')
ax1.grid(True, alpha=0.3)

# NPV by initial rate and price
for price in price_scenarios:
    subset = sensitivity[sensitivity['price'] == price]
    ax2.plot(subset['qi'], subset['NPV'], 'o-', label=f'${price}/bbl', linewidth=2, markersize=6)

ax2.axhline(y=0, color='r', linestyle='--', linewidth=2, alpha=0.5)
ax2.set_xlabel('Initial Rate (qi) bbl/month', fontsize=12)
ax2.set_ylabel('NPV ($)', fontsize=12)
ax2.set_title('NPV Sensitivity', fontsize=14, fontweight='bold')
ax2.legend(title='Oil Price')
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

plt.tight_layout()
plt.show()

## Summary

In this notebook, we:
1. ✓ Calculated EUR (Estimated Ultimate Recovery)
2. ✓ Computed NPV and cash flows
3. ✓ Determined payback period
4. ✓ Performed price sensitivity analysis
5. ✓ Ran multi-parameter sensitivity analysis

## Key Insights

- EUR depends on economic limit and decline parameters
- NPV is highly sensitive to oil price
- Payback period helps assess investment risk
- Sensitivity analysis quantifies uncertainty

## Next Steps

- **Notebook 03**: Analyze multiple wells simultaneously
- **Notebook 04**: Explore advanced ML forecasting